# Analysis of Portenga and Bierman (2011) supplementary data

In [30]:
import pandas as pd
import os
import numpy as np
folder ='/Users/gr_1/Documents/cosmo_predict/'
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import sklearn

## step 1 Load excel sheet, look at basin averaged rates (sheets 4 - 12), concatenate

In [31]:

a = pd.DataFrame()
for i in range(4,12):
    b = pd.read_excel(folder + 'rates_cosmo.xlsx',sheet_name = i)
    if i < 4:
        b['type'] = 'bedrock'
    else:
        b['type'] = 'basinwide'
    a = a.append(b)
a.index = range(0,len(a))

AttributeError: 'DataFrame' object has no attribute 'append'

## Some of the data labels are messed up, correct

In [32]:
for col in a.columns:
    for i in range(len(a)):
        if len(str(a[col][i:i+1].values[0]).split("\n"))>1:
            a.loc[i,col] = str(a[col][i:i+1].values[0]).split('\n')[0]
    if 'Unnamed' in str(col):
        a = a.drop(col,1)
for s in range(len(a)):
    a.loc[s,'Climate Zone (Sub-category)'] = str(a.loc[s,'Climate Zone (Sub-category)']).lower()

## Want to discretize continuous data for the seaborn box plots

In [33]:
a_discrete = pd.DataFrame()
for col in a.columns:
    try:
        cval = pd.to_numeric(a[col],errors='coerce')
        a_discrete[col+' binned'] = pd.cut(cval,bins = np.arange(min(cval),max(cval),(max(cval)-min(cval))/8))
    except:
        a_discrete[col] = a[col]
    
a_discrete['CRONUS Erosion Rate (m My)'] = pd.to_numeric(a['CRONUS Erosion Rate (m My)'],errors='coerce')

KeyError: 'CRONUS Erosion Rate (m My)'

## Box plots of excel variables vs. Cronus erosion rates

In [20]:
if 1:
    c = 0
    for col in np.flip(a_discrete.columns[3:-5]):

        fig_ax = plt.subplots(figsize=(10,5))
        g = sns.boxplot(data = a_discrete,y='CRONUS Erosion Rate (m My)',x=col)
        plt.xticks(rotation=80)
        g.set_yscale('log')
        c += 1

## Random forest regression

In [22]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from sklearn.svm import SVR
from sklearn import impute
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [23]:
a['dist'] = np.sqrt(np.float64(a['Longitude (decimal degrees)'])**2 + np.float64(a['Latitude (decimal degrees)'])**2)
a['Longitude (decimal degrees)'] = np.float64(a['Longitude (decimal degrees)'])

KeyError: 'Longitude (decimal degrees)'

In [ ]:
a['dist_bin'] = pd.factorize(pd.cut(pd.to_numeric(a['Longitude (decimal degrees)']),50))[0]

In [ ]:
a2 = pd.DataFrame()
for i,data in a.groupby('dist_bin'):  
    if len(data) > 0:
        a3 = data.mean()
        for col in a.columns:
            if (a[col].dtype != np.number):
                a3[col] = data.iloc[0][col]
        a2 = a2.append(a3,ignore_index=True)

#a = a2

In [ ]:
a = a.sort_values('Longitude (decimal degrees)')
#a=a[::-1]

## Data prep

In [24]:
#a = a[::-1]
a_rf = pd.DataFrame()
for col in a.columns[4:-3]:
    try:

        if ('Latitude' in col) | ('Longitude' in col):
            continue
        cval = pd.to_numeric(a[col],errors='coerce')
       # print(max(cval))

        pd.cut(cval,bins = np.arange(min(cval),max(cval),(max(cval)-min(cval))/8))

        a_rf[col] = cval
    except Exception as e:
        print(e)
        le = sklearn.preprocessing.LabelEncoder()
        le.fit(a[col].astype('str').values)
        l = le.transform(a[col].astype('str').values) 
        a_rf[col] = l

    
a_rf['CRONUS Erosion Rate (m My)'] = pd.to_numeric(a['CRONUS Erosion Rate (m My)'],errors='coerce')

a_rf = a_rf.drop(np.where(a['Mean Basin Elevation (m)'].isna())[0])

KeyError: 'CRONUS Erosion Rate (m My)'

In [12]:
if 0:
    a_rf.index = range(len(a_rf))
    lencvs = int(np.floor(len(a_rf)/ncv))
    test_ids = []
    a_rf2 = pd.DataFrame()
    for i in range(ncv):
        if i < ncv-1:
            a_rf_sub = a_rf.iloc[i * lencvs : (i+1) *lencvs]
        else:
            a_rf_sub = a_rf.iloc[i * lencvs ::]
        #test_ids.append(a_rf_sub.sample(frac=1).index)
        a_rf2 = a_rf2.append(a_rf_sub.sample(frac=1))
    a_rf = a_rf2

In [13]:
plt.plot(a_rf['Mean Basin Slope (°)'],a_rf['CRONUS Erosion Rate (m My)'],'.')
plt.xscale('log')
plt.yscale('log')

KeyError: 'Mean Basin Slope (°)'

# Cross validation - test dataset vs estimate with random forests

In [25]:
a_rf = a_rf.sample(frac=1)
a_rf.index = range(len(a_rf))
ncv = 5
selected_vars = ['Mean Basin Elevation (m)','Basin Relief (m)',
                'Climate Zone (Main)','Mean Basin Slope (°)','Mean Annual Precipitation (mm yr)',
                'Mean Annual Temperature (°C)','Seismicity','Seismic Regime']
#selected_vars = ['Mean Basin Slope (°)']
try:
    a_rf = a_rf.drop('CRONUS Erosion Rate Error (m My)',1)
except:
    print('here')
predictors = a_rf.drop('CRONUS Erosion Rate (m My)',1)

targets = a_rf['CRONUS Erosion Rate (m My)']
predictors.index = range(len(predictors))
targets.index = range(len(predictors))
a_rf.index = range(len(a_rf))
select = False
ero_thres = False
if select:
    predictors = predictors[selected_vars]
if ero_thres:
    
        drop_rows = np.where((targets>1000)|(targets<10))[0]
        for drop_row in drop_rows:
            predictors = predictors.drop(drop_row)
            targets = targets.drop(drop_row)

model = RandomForestRegressor()#max_depth=50,criterion='mse',bootstrap = 'True',min_samples_split=50)
#model = LinearRegression()
seed = 5
cv = model_selection.KFold(n_splits=ncv)
a_rf.index = range(len(a_rf))
c=0
mnr2=0
mnmn =0
for train_index, test_index in cv.split(predictors):
    X_train, X_test = predictors.iloc[train_index].values, predictors.iloc[test_index].values
    X_train[X_train>5000000] = np.nan
    y_train, y_test = targets.iloc[train_index].values, targets.iloc[test_index].values
    y_train,y_test = y_train.reshape(-1,1),y_test.reshape(-1,1)
    
    ## Test data is training data - see how much overfitting occurs
    #X_test = X_train.copy()
    #y_test = y_train.copy()

    imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_train)
    X_train = imp.transform(X_train)
    imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(X_test)
    X_test = imp.transform(X_test)
    imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(y_train)
    y_train = imp.transform(y_train).ravel()
    imp = impute.SimpleImputer(missing_values=np.nan, strategy='mean')
    imp.fit(y_test)
    y_test = imp.transform(y_test).ravel()
    # For training, fit() is used
    scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
    X_train = scaling.transform(X_train)
    X_test = scaling.transform(X_test)
    model.fit(X_train, y_train)

    # Default metric is R2 for regression, which can be accessed by score()
    model.score(X_test, y_test)

    # For other metrics, we need the predictions of the model
    y_pred = model.predict(X_test)
    fig,ax = plt.subplots(figsize=(5,5))
    plt.plot(y_test,y_pred,'.')
    ax.set_yscale('log')
    ax.set_xscale('log')
    plt.xlabel('Observed m/Myr')
    plt.ylabel('Estimated m/Myr')
    print('Cross-validation # ' + str(c))
    #print('Median error = ', metrics.median_absolute_error(y_test, y_pred))

    y_test_n = y_test[np.logical_and(y_test>0,y_pred>0)]
    y_pred_n = y_pred[np.logical_and(y_pred>0,y_test>0)]
    r2 = metrics.r2_score(np.log(y_test_n),np.log(y_pred_n))
    mn = metrics.median_absolute_error(np.log(y_test_n), np.log(y_pred_n))
    print('r2 = ',r2)
    mnmn+=mn
    mnr2+=r2
    c+=1
    plt.xlim([0 ,1000])
    plt.ylim([0 ,1000])
print(mnr2/c)
print(mnmn/c)

here


TypeError: drop() takes from 1 to 2 positional arguments but 3 were given

## Baseline

In [26]:
from sklearn.linear_model import LinearRegression
c=0
mnr2=0
mnmn = 0
cv = model_selection.KFold(n_splits=ncv)

for train_index, test_index in cv.split(predictors):
    mod = LinearRegression()
    X_train, X_test = predictors.iloc[train_index].values, predictors.iloc[test_index].values
    y_train, y_test = targets.iloc[train_index].values, targets.iloc[test_index].values
    
    X_train = X_train[:,4]
    X_test = X_test[:,4]
    isn = np.where(np.logical_not(np.isnan(X_train)))
    isn2 = np.where(np.logical_not((np.isnan(X_test))))
    mod = mod.fit(X_train[isn].reshape(-1,1),y_train[isn].reshape(-1,1))
    
    p = mod.predict((X_test[isn2].reshape(-1,1)))
    #e = np.mean((p - np.log(y_train[isn2]))**2)
    #p = np.exp(p) * 1#np.exp(e/2)
    fig,ax = plt.subplots(figsize=(5,5))
    #plt.plot(x,p,markersize=10)
    #plt.plot(a_rf[a_rf.columns[i]], np.float64(a_rf['CRONUS Erosion Rate (m My)']),'.')
    plt.plot(y_test[isn2],p,'.')
    r2 = metrics.r2_score(y_test[isn2],p)
    mn = metrics.median_absolute_error(y_test[isn2], p)

    mnr2 += r2
    mnmn += mn
    ax.set_yscale('log')
    ax.set_xscale('log')
    c+=1
print(mnr2/c)
print(mnmn/c)


    #ax.set_ylim([.])


NameError: name 'predictors' is not defined

In [27]:
predictors2 = predictors.copy()#.sample(frac=1)
targets2 = targets[predictors2.index]

NameError: name 'predictors' is not defined

In [28]:
from sklearn.linear_model import LinearRegression
mod = LinearRegression()
targeti = targets2.copy()
predicti = predictors2.copy()

ti = sklearn.impute.SimpleImputer()
pi = sklearn.impute.SimpleImputer()

ti.fit(targeti.to_numpy().reshape(-1,1))
targeti = ti.transform(targeti.to_numpy().reshape(-1,1))

pi.fit(predicti.to_numpy())
predicti = pi.transform(predicti.to_numpy())
#x[x<=0] = 1e-10X
x=predicti#prednp.abs(X_train[:,4].reshape(-1,1))
y_train=targeti
y_test = targeti
x=(x)

mod = mod.fit((x),(y_train))
p = mod.predict((x))
fig,ax = plt.subplots(figsize=(7,5))
yt = y_train.copy()
yt = yt[p>0]
#p = p[p>0]
#plt.plot(x,p,markersize=10)
#plt.plot(a_rf[a_rf.columns[i]], np.float64(a_rf['CRONUS Erosion Rate (m My)']),'.')
x = x[:,4]
plt.plot(np.log(p),np.log(yt),'.')
#plt.plot((x),p,'.')
#plt.plot(x)
#plt.xscale('log')
#plt.yscale('log')
print(metrics.r2_score(np.log(yt),np.log(p))

#ax.set_ylim([.])
#ax.set_xlabel(a_rf.columns[i])

SyntaxError: unexpected EOF while parsing (2680198704.py, line 37)

In [29]:
from sklearn.linear_model import LinearRegression
mod = LinearRegression()
cv = model_selection.KFold(n_splits=5)

for train_index, test_index in cv.split(predictors2):
    targeti = targets.copy().loc[train_index]
    predicti = predictors2.copy().loc[train_index]
    
    targett = targets.copy().loc[test_index]
    predictt = predictors.copy().loc[test_index]

    ti = sklearn.impute.SimpleImputer()
    pi = sklearn.impute.SimpleImputer()
    tt = sklearn.impute.SimpleImputer()
    pt = sklearn.impute.SimpleImputer()

    ti.fit(targeti.to_numpy().reshape(-1,1))
    targeti = ti.transform(targeti.to_numpy().reshape(-1,1))

    pi.fit(predicti.to_numpy())
    predicti = pi.transform(predicti.to_numpy())
    
    
    tt.fit(targett.to_numpy().reshape(-1,1))
    targett = tt.transform(targett.to_numpy().reshape(-1,1))

    pt.fit(predictt.to_numpy())
    predictt = pt.transform(predictt.to_numpy())
    #targett=targeti.copy()
    #predictt=predicti.copy()
    #x[x<=0] = 1e-10X
    x=predicti#prednp.abs(X_train[:,4].reshape(-1,1))
    y_train=targeti
    y_test = targett
    xt=predictt
    

    mod = mod.fit((x),np.log(y_train))
    p = mod.predict(xt)
    fig,ax = plt.subplots(figsize=(7,5))
    yt = y_train.copy()
    #yt = yt[p>0]
    #p = p[p>0]
    #plt.plot(x,p,markersize=10)
    #plt.plot(a_rf[a_rf.columns[i]], np.float64(a_rf['CRONUS Erosion Rate (m My)']),'.')
    plt.plot(p,np.log(y_test),'.')
    #plt.plot((x),p,'.')
    #plt.plot(x)
    #plt.xscale('log')
    #plt.yscale('log')
    print(metrics.r2_score(np.log(y_test),p))

#ax.set_ylim([.])
#ax.set_xlabel(a_rf.columns[i])

NameError: name 'predictors2' is not defined